<a href="https://colab.research.google.com/github/sujansuprajn/Hackathons/blob/main/Loan_approval_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

# Setup Kaggle API key configuration
kaggle_path = os.path.expanduser('~/.kaggle/kaggle.json')
if not os.path.exists(kaggle_path):
    os.makedirs(os.path.dirname(kaggle_path), exist_ok=True)
    !cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
else:
    print("Kaggle configuration already set up.")

Mounted at /content/drive


In [2]:
#dataset download
!kaggle competitions download -c playground-series-s4e10

#unzip
!unzip playground-series-s4e10.zip
!kaggle datasets download -d chilledwanker/loan-approval-prediction
!unzip loan-approval-prediction.zip

  0% 0.00/1.45M [00:00<?, ?B/s]
100% 1.45M/1.45M [00:00<00:00, 118MB/s]
Archive:  playground-series-s4e10.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
Dataset URL: https://www.kaggle.com/datasets/chilledwanker/loan-approval-prediction
License(s): apache-2.0
  0% 0.00/368k [00:00<?, ?B/s]
100% 368k/368k [00:00<00:00, 75.9MB/s]
Archive:  loan-approval-prediction.zip
  inflating: credit_risk_dataset.csv  


In [47]:
#importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer, RobustScaler, OneHotEncoder
from xgboost import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler

In [68]:
df = pd.read_csv('train.csv', index_col='id')
X_test_full = pd.read_csv('test.csv', index_col='id')

In [54]:
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
id,,,,,,,,,,,,
0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [69]:
original = pd.read_csv('credit_risk_dataset.csv')
X= pd.concat([df, original],ignore_index=True)
print("shape of the data :",X.shape)

shape of the data : (91226, 12)


In [70]:
#removing rows with missing taret
X.dropna(axis=0, subset=['loan_status'], inplace=True)
y = X.loan_status

X['loan_relative_income_amount'] = X['loan_amnt'] * X['person_income']
X['loan_percent_income'] = X['loan_percent_income'] * 100
X.drop(['loan_status','loan_amnt','person_age', 'person_income'],axis=1, inplace=True)

X_test_full['loan_relative_income_amount'] = X_test_full['loan_amnt'] * X_test_full['person_income']
X_test_full['loan_percent_income'] = X_test_full['loan_percent_income'] * 100
X_test_full.drop(['loan_amnt','person_age', 'person_income'],axis=1, inplace=True)


In [71]:
#break off validation data from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state=0)

# categorical cols with low cardinality
cat_cols = [cname for cname in X_train_full.columns
            if X_train_full[cname].dtype == 'object' and
            X_train_full[cname].nunique() < 10]

# numerical cols
num_cols = [cname for cname in X_train_full.columns
            if X_train_full[cname].dtype in ['int64','float64']]


# keeping only selected cols
my_cols = cat_cols + num_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

for col in cat_cols:
    X_train[col] = X_train[col].astype('category')
    X_valid[col] = X_valid[col].astype('category')
    X_test[col] = X_test[col].astype('category')

In [72]:
my_cols

['person_home_ownership',
 'loan_intent',
 'loan_grade',
 'cb_person_default_on_file',
 'person_emp_length',
 'loan_int_rate',
 'loan_percent_income',
 'cb_person_cred_hist_length',
 'loan_relative_income_amount']

PreProcessing With Pipeline

In [78]:
# preprocessing numerical data
numerical_transformer = SimpleImputer(strategy='median')


# preprocessing categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value = 'missing')),
    ('onehot', OneHotEncoder(drop= 'first', handle_unknown='ignore', sparse_output=False))
])

# bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
   ])


model = XGBClassifier(
    max_depth=3,
    learning_rate=0.08683740317846539,
    n_estimators=961,
    random_state=0,
    enable_categorical=True,
    gamma=0.5,
    subsample=0.9292131415430409,
    scale_pos_weight= 6,
    early_stopping_rounds=10,
    colsample_bytree= 0.90987893726981,
    reg_alpha=0.8570404246557465,
    reg_lambda=5.995860925165919e-05)

#Best parameters: {'max_depth': 3, 'learning_rate': 0.08683740317846539, 'n_estimators': 961, 'subsample': 0.9292131415430409, 'colsample_bytree': 0.90987893726981, 'scale_pos_weight': 6.971333166633001, 'lambda': 5.995860925165919e-05, 'alpha': 0.8570404246557465}
#Best ROC AUC score: 0.9561823463359602
#earlier depth6, learning_rate 0.01, estimators 1000, scaloe pos wieth 5, alpha= 0.5

Predicting the Target Variable

In [79]:
# Bundling preprocessing and modeling in Pipeline
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
  ])

# Preprocessing of training data, fit model
my_pipeline.fit(X_train, y_train, model__verbose=False, model__eval_set=[(preprocessor.fit_transform(X_train), y_train)])


preds = my_pipeline.predict(X_valid)
preds_proba = my_pipeline.predict_proba(X_valid)[:, 1]

xgb_auc = roc_auc_score(y_valid, preds_proba)
print(f"AUC: {xgb_auc:.4f}")

print("Confusion Matrix:")
print(confusion_matrix(y_valid, preds))
print("\nClassification Report:")
print(classification_report(y_valid, preds))

AUC: 0.9320
Confusion Matrix:
[[13885  1283]
 [  605  2473]]

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.92      0.94     15168
           1       0.66      0.80      0.72      3078

    accuracy                           0.90     18246
   macro avg       0.81      0.86      0.83     18246
weighted avg       0.91      0.90      0.90     18246



In [80]:
# preprocessing test data and predicting
preds_test = my_pipeline.predict(X_test)
output = pd.DataFrame({'id':X_test.index,
                       'loan_status':preds_test})
output.to_csv('submission.csv', index = False)

!kaggle competitions submit -c playground-series-s4e10 -f submission.csv -m "Message"

100% 305k/305k [00:00<00:00, 509kB/s]
Successfully submitted to Loan Approval Prediction